In [1]:
# Импорты и настройка логгирования
from pathlib import Path

import polars as pl

from src.dataset_manager import create_dataset_combinations, create_report, get_dataset_statistics, save_datasets
from src.descriptor_calculator import (
    calculate_fingerprints,
    calculate_mordred_descriptors,
    calculate_padel_descriptors,
    calculate_rdkit_descriptors,
)
from src.descriptor_visualization import create_descriptors_visualization, create_summary_dashboard
from src.feature_filter import filter_descriptors, filter_fingerprints

# Наши модули из src/
from src.logging_config import get_logger, setup_logging

# Настройка логгирования
# Настройка логгинга
setup_logging(level="INFO", log_file="./data/descriptor_calculation.log")
logger = get_logger(__name__)
# Проверяем доступность библиотек
try:
    import rdkit

    logger.info("Результат: RDKit доступен")
except ImportError:
    logger.error("Ошибка: RDKit не найден")

try:
    import mordred

    logger.info("Результат: Mordred доступен")
except ImportError:
    logger.warning("Предупреждение: Mordred не найден (опционально)")

try:
    import padelpy

    logger.info("Результат: PaDELPy доступен")
except ImportError:
    logger.warning("Предупреждение: PaDELPy не найден (опционально)")

try:
    import plotly

    logger.info("Результат: Plotly доступен")
except ImportError:
    logger.error("Ошибка: Plotly не найден")

# Настройка путей
DATA_DIR = Path("data")
OUTPUT_DIR = Path("data/descriptors")
VISUALIZATION_DIR = Path("data/descriptors/visualizations")

logger.info(f"Рабочие директории: {DATA_DIR}, {OUTPUT_DIR}, {VISUALIZATION_DIR}")

2025-07-09 23:22:38,236 - __main__ - INFO - Результат: RDKit доступен
2025-07-09 23:22:38,236 - __main__ - INFO - Результат: Mordred доступен
2025-07-09 23:22:38,237 - __main__ - INFO - Результат: PaDELPy доступен
2025-07-09 23:22:38,237 - __main__ - INFO - Результат: Plotly доступен
2025-07-09 23:22:38,237 - __main__ - INFO - Рабочие директории: data, data/descriptors, data/descriptors/visualizations


In [2]:
# Загрузка очищенного датасета из мини-таска 1
logger.info("Файл: Загрузка очищенного датасета COX-2...")

# Попробуем загрузить из Parquet (быстрее), если нет - из CSV
parquet_path = DATA_DIR / "cox2_final_dataset.parquet"
csv_path = DATA_DIR / "cox2_final_dataset.csv"

if parquet_path.exists():
    dataset = pl.read_parquet(parquet_path)
    logger.info(f"Результат: Загружен парquet файл: {dataset.shape}")
elif csv_path.exists():
    dataset = pl.read_csv(csv_path)
    logger.info(f"Результат: Загружен CSV файл: {dataset.shape}")
else:
    raise FileNotFoundError("Не найден очищенный датасет! Сначала выполните мини-таск 1")

# Проверяем структуру данных
logger.info(f"Колонки датасета: {dataset.columns}")
logger.info(f"Количество молекул: {dataset.shape[0]}")

# Проверяем наличие необходимых колонок (исправленные имена)
required_columns = ["canonical_smiles", "pic50"]
for col in required_columns:
    if col not in dataset.columns:
        raise ValueError(f"Отсутствует обязательная колонка: {col}")

# Базовые данные для объединения с дескрипторами
base_data = dataset.select(["canonical_smiles", "pic50"])
smiles_list = dataset["canonical_smiles"].to_list()

logger.info(f"Результат: Подготовлено {len(smiles_list)} SMILES для расчёта дескрипторов")

2025-07-09 23:22:38,241 - __main__ - INFO - Файл: Загрузка очищенного датасета COX-2...
2025-07-09 23:22:38,267 - __main__ - INFO - Результат: Загружен парquet файл: (3661, 48)
2025-07-09 23:22:38,267 - __main__ - INFO - Колонки датасета: ['molecule_chembl_id', 'canonical_smiles', 'standard_type', 'standard_value', 'standard_units', 'pchembl_value', 'activity_id', 'activity_properties', 'assay_chembl_id', 'assay_description', 'assay_type', 'bao_endpoint', 'bao_format', 'bao_label', 'document_chembl_id', 'document_journal', 'document_year', 'ligand_efficiency', 'molecule_pref_name', 'parent_molecule_chembl_id', 'potential_duplicate', 'qudt_units', 'record_id', 'relation', 'src_id', 'standard_flag', 'standard_relation', 'target_chembl_id', 'target_organism', 'target_pref_name', 'target_tax_id', 'type', 'units', 'uo_units', 'value', 'standard_value_nm', 'standard_units_standardized', 'mol_weight', 'log_p', 'hbd', 'hba', 'tpsa', 'rotatable_bonds', 'aromatic_rings', 'is_valid_molecule', 'ac

In [3]:
# Расчёт молекулярных дескрипторов

# 1. RDKit дескрипторы (всегда доступны)
logger.info("Расчет: Расчёт RDKit дескрипторов...")
rdkit_descriptors = calculate_rdkit_descriptors(smiles_list)
logger.info(f"Результат: RDKit дескрипторы: {rdkit_descriptors.shape}")

# 2. Mordred дескрипторы (если доступны)
logger.info("Расчет: Расчёт Mordred дескрипторов...")
mordred_descriptors = calculate_mordred_descriptors(smiles_list)
if mordred_descriptors is not None:
    logger.info(f"Результат: Mordred дескрипторы: {mordred_descriptors.shape}")
else:
    logger.warning("Предупреждение: Mordred дескрипторы недоступны")

# Сохраняем исходные дескрипторы для статистики
original_descriptors = {"rdkit": rdkit_descriptors}
if mordred_descriptors is not None:
    original_descriptors["mordred"] = mordred_descriptors

2025-07-09 23:22:38,273 - __main__ - INFO - Расчет: Расчёт RDKit дескрипторов...
2025-07-09 23:22:38,282 - src.descriptor_calculator - INFO - Загружены rdkit дескрипторы из кэша (217 дескрипторов)
2025-07-09 23:22:38,282 - __main__ - INFO - Результат: RDKit дескрипторы: (3661, 217)
2025-07-09 23:22:38,283 - __main__ - INFO - Расчет: Расчёт Mordred дескрипторов...
2025-07-09 23:22:38,330 - src.descriptor_calculator - INFO - Загружены mordred дескрипторы из кэша (1613 дескрипторов)
2025-07-09 23:22:38,330 - __main__ - INFO - Результат: Mordred дескрипторы: (3661, 1613)


In [4]:
# 3. PaDEL дескрипторы (если доступны)
logger.info("Расчет: Расчёт PaDEL дескрипторов...")
try:
    # Рассчитываем PaDEL дескрипторы для всех молекул
    padel_descriptors = calculate_padel_descriptors(
        smiles_list,
        fingerprints=True,  # Включаем фингерпринты
        d_2d=True,  # Включаем 2D дескрипторы
        d_3d=True,  # Отключаем 3D дескрипторы (быстрее)
    )

    if padel_descriptors is not None:
        logger.info(f"Результат: PaDEL дескрипторы: {padel_descriptors.shape}")
        logger.info(f"Метка: Примеры дескрипторов: {padel_descriptors.columns[:10]}")

        # Добавляем PaDEL к исходным дескрипторам
        original_descriptors["padel"] = padel_descriptors
    else:
        logger.warning("PaDEL дескрипторы недоступны")

except Exception as e:
    logger.error(f"Ошибка при расчёте PaDEL дескрипторов: {e}")
    logger.warning("Продолжаем без PaDEL дескрипторов")

2025-07-09 23:22:38,334 - __main__ - INFO - Расчет: Расчёт PaDEL дескрипторов...
2025-07-09 23:22:38,388 - src.descriptor_calculator - INFO - Загружены padel дескрипторы из кэша (2325 дескрипторов)
2025-07-09 23:22:38,388 - __main__ - INFO - Результат: PaDEL дескрипторы: (3661, 2325)
2025-07-09 23:22:38,389 - __main__ - INFO - Метка: Примеры дескрипторов: ['nAcid', 'ALogP', 'ALogp2', 'AMR', 'apol', 'naAromAtom', 'nAromBond', 'nAtom', 'nHeavyAtom', 'nH']


In [5]:
# Расчёт молекулярных фингерпринтов
logger.info("Фильтрация: Расчёт молекулярных фингерпринтов...")
fingerprint_dfs = calculate_fingerprints(smiles_list)

# Выводим информацию о рассчитанных фингерпринтах
for fp_type, fp_df in fingerprint_dfs.items():
    logger.info(f"Результат: {fp_type}: {fp_df.shape}")

# Добавляем фингерпринты к исходным дескрипторам для статистики
original_descriptors.update(fingerprint_dfs)

2025-07-09 23:22:38,392 - __main__ - INFO - Фильтрация: Расчёт молекулярных фингерпринтов...
2025-07-09 23:22:38,427 - src.descriptor_calculator - INFO - Загружены fingerprint_morgan_2048 дескрипторы из кэша (2048 дескрипторов)
2025-07-09 23:22:38,447 - src.descriptor_calculator - INFO - Загружены fingerprint_morgan_1024 дескрипторы из кэша (1024 дескрипторов)
2025-07-09 23:22:38,451 - src.descriptor_calculator - INFO - Загружены fingerprint_maccs дескрипторы из кэша (166 дескрипторов)
2025-07-09 23:22:38,473 - src.descriptor_calculator - INFO - Загружены fingerprint_atompairs дескрипторы из кэша (2048 дескрипторов)
2025-07-09 23:22:38,498 - src.descriptor_calculator - INFO - Загружены fingerprint_topological дескрипторы из кэша (2048 дескрипторов)
2025-07-09 23:22:38,498 - src.descriptor_calculator - INFO - Кэш: Все фингерпринты загружены из кэша
2025-07-09 23:22:38,499 - __main__ - INFO - Результат: morgan_2048: (7322, 2048)
2025-07-09 23:22:38,499 - __main__ - INFO - Результат: morg

In [6]:
# Фильтрация признаков по критериям качества

logger.info("Фильтрация: Фильтрация дескрипторов и фингерпринтов...")

# 1. Фильтрация RDKit дескрипторов
rdkit_filtered = filter_descriptors(
    rdkit_descriptors,
    name_prefix="RDKit",
    null_threshold=0.1,  # Удаляем дескрипторы с >10% null
    variance_threshold=0.001,  # Удаляем дескрипторы с очень низкой дисперсией
    correlation_threshold=0.7,  # Удаляем высококоррелированные (r>0.7)
)

# 2. Фильтрация Mordred дескрипторов (если доступны)
if mordred_descriptors is not None:
    mordred_filtered = filter_descriptors(
        mordred_descriptors, name_prefix="Mordred", null_threshold=0.1, variance_threshold=0.001, correlation_threshold=0.7
    )
else:
    mordred_filtered = None

# 3. Фильтрация фингерпринтов (удаляем неинформативные биты)
filtered_fingerprints = {}
for fp_type, fp_df in fingerprint_dfs.items():
    filtered_fp = filter_fingerprints(
        fp_df,
        name_prefix=fp_type,
        variance_threshold=0.01,  # Для бинарных данных используем более низкий порог
    )
    filtered_fingerprints[fp_type] = filtered_fp

# Сохраняем отфильтрованные дескрипторы для статистики
filtered_descriptors = {"rdkit": rdkit_filtered}
if mordred_filtered is not None:
    filtered_descriptors["mordred"] = mordred_filtered

filtered_descriptors.update(filtered_fingerprints)

logger.info("Фильтрация завершена")

2025-07-09 23:22:38,504 - __main__ - INFO - Фильтрация: Фильтрация дескрипторов и фингерпринтов...
2025-07-09 23:22:38,504 - src.feature_filter - INFO - Фильтрация дескрипторов RDKit...
2025-07-09 23:22:38,504 - src.feature_filter - INFO - Исходное количество дескрипторов: 217
2025-07-09 23:22:38,513 - src.feature_filter - INFO - Удалено дескрипторов с >10.0% null: 0
2025-07-09 23:22:38,553 - src.feature_filter - INFO - Удалено дескрипторов с низкой дисперсией (<0.001): 17
2025-07-09 23:22:38,612 - src.feature_filter - INFO - Удалено высококоррелированных дескрипторов (r>0.7): 75
2025-07-09 23:22:38,613 - src.feature_filter - INFO - Финальное количество дескрипторов RDKit: 125
2025-07-09 23:22:38,614 - src.feature_filter - INFO - Фильтрация дескрипторов Mordred...
2025-07-09 23:22:38,614 - src.feature_filter - INFO - Исходное количество дескрипторов: 1613
2025-07-09 23:22:38,783 - src.feature_filter - INFO - Удалено дескрипторов с >10.0% null: 184
2025-07-09 23:22:43,165 - src.feature_

In [7]:
# Создание различных комбинаций датасетов

logger.info("Статистика: Создание различных комбинаций датасетов...")

# Создаём различные комбинации признаков для мини-таска 3
datasets_to_save = create_dataset_combinations(
    base_data=base_data,
    rdkit_filtered=rdkit_filtered,
    mordred_filtered=mordred_filtered,
    filtered_fingerprints=filtered_fingerprints,
)

# Получаем статистику по созданным датасетам
dataset_statistics = get_dataset_statistics(datasets_to_save)

# Выводим краткую статистику
logger.info("График: Статистика созданных датасетов:")
for name, stats in dataset_statistics.items():
    logger.info(f"  {name}: {stats['molecules']} молекул × {stats['features']} признаков")

logger.info(f"Создано {len(datasets_to_save)} датасетов для мини-таска 3")

2025-07-09 23:22:46,401 - __main__ - INFO - Статистика: Создание различных комбинаций датасетов...
2025-07-09 23:22:46,402 - src.dataset_manager - INFO - Успешно: RDKit датасет: (3661, 127)
2025-07-09 23:22:46,403 - src.dataset_manager - INFO - Успешно: Mordred датасет: (3661, 193)
2025-07-09 23:22:46,405 - src.dataset_manager - INFO - Успешно: morgan_2048 датасет: (7322, 365)
2025-07-09 23:22:46,406 - src.dataset_manager - INFO - Успешно: morgan_1024 датасет: (7322, 412)
2025-07-09 23:22:46,407 - src.dataset_manager - INFO - Успешно: maccs датасет: (3661, 2)
2025-07-09 23:22:46,407 - src.dataset_manager - INFO - Успешно: atompairs датасет: (3661, 2)
2025-07-09 23:22:46,407 - src.dataset_manager - INFO - Успешно: topological датасет: (3661, 2)
2025-07-09 23:22:46,409 - src.dataset_manager - INFO - Успешно: Комбинированный датасет (RDKit + Morgan): (7322, 537)
2025-07-09 23:22:46,416 - src.dataset_manager - INFO - Успешно: Лучший комбинированный датасет: (7322, 562)
2025-07-09 23:22:46,

In [8]:
# Сохранение датасетов и создание отчёта

logger.info("Сохранено: Сохранение датасетов...")

# Создаём директорию для выходных файлов
OUTPUT_DIR.mkdir(exist_ok=True)

# Сохраняем все датасеты в CSV и Parquet форматах
save_datasets(datasets_to_save, OUTPUT_DIR)

# Создаём сводный отчёт
report_path = OUTPUT_DIR / "cox2_descriptors_report.txt"
create_report(
    datasets=datasets_to_save,
    smiles_list=smiles_list,
    rdkit_descriptors=rdkit_descriptors,
    rdkit_filtered=rdkit_filtered,
    mordred_descriptors=mordred_descriptors,
    mordred_filtered=mordred_filtered,
    fingerprint_dfs=fingerprint_dfs,
    filtered_fingerprints=filtered_fingerprints,
    output_path=report_path,
)

logger.info(f"Все датасеты сохранены в {OUTPUT_DIR}")
logger.info(f"Отчёт сохранён: {report_path}")

2025-07-09 23:22:46,429 - __main__ - INFO - Сохранено: Сохранение датасетов...
2025-07-09 23:22:46,448 - src.dataset_manager - INFO - Сохранено: Сохранён датасет rdkit_descriptors: (3661, 127)
2025-07-09 23:22:46,483 - src.dataset_manager - INFO - Сохранено: Сохранён датасет mordred_descriptors: (3661, 193)
2025-07-09 23:22:46,506 - src.dataset_manager - INFO - Сохранено: Сохранён датасет fingerprints_morgan_2048: (7322, 365)
2025-07-09 23:22:46,531 - src.dataset_manager - INFO - Сохранено: Сохранён датасет fingerprints_morgan_1024: (7322, 412)
2025-07-09 23:22:46,533 - src.dataset_manager - INFO - Сохранено: Сохранён датасет fingerprints_maccs: (3661, 2)
2025-07-09 23:22:46,534 - src.dataset_manager - INFO - Сохранено: Сохранён датасет fingerprints_atompairs: (3661, 2)
2025-07-09 23:22:46,536 - src.dataset_manager - INFO - Сохранено: Сохранён датасет fingerprints_topological: (3661, 2)
2025-07-09 23:22:46,575 - src.dataset_manager - INFO - Сохранено: Сохранён датасет combined_rdkit_mo

In [9]:
# Создание визуализаций и анализ

logger.info("Создание визуализаций...")

# Создаём директорию для визуализаций
VISUALIZATION_DIR.mkdir(exist_ok=True)

# Создаём комплексную визуализацию дескрипторов
create_descriptors_visualization(
    datasets=datasets_to_save,
    original_descriptors=original_descriptors,
    filtered_descriptors=filtered_descriptors,
    output_dir=VISUALIZATION_DIR,
)

# Создаём сводную панель статистики
dashboard_path = VISUALIZATION_DIR / "summary_dashboard.html"
create_summary_dashboard(datasets=datasets_to_save, statistics=dataset_statistics, output_path=dashboard_path)

logger.info(f"Все визуализации сохранены в {VISUALIZATION_DIR}")

2025-07-09 23:22:46,633 - __main__ - INFO - Создание визуализаций...
2025-07-09 23:22:46,668 - src.descriptor_visualization - INFO - Успешно: График количества признаков сохранён
2025-07-09 23:22:47,411 - src.descriptor_visualization - INFO - Успешно: Анализ пропущенных значений сохранён
2025-07-09 23:22:47,427 - src.descriptor_visualization - INFO - Успешно: График размеров датасетов сохранён
2025-07-09 23:22:47,659 - src.descriptor_visualization - INFO - Успешно: Анализ фингерпринтов сохранён
2025-07-09 23:22:47,660 - src.descriptor_visualization - INFO - Статистика: Визуализации сохранены в data/descriptors/visualizations
2025-07-09 23:22:47,672 - src.descriptor_visualization - INFO - Успешно: Сводная панель сохранена: data/descriptors/visualizations/summary_dashboard.html
2025-07-09 23:22:47,673 - __main__ - INFO - Все визуализации сохранены в data/descriptors/visualizations
